In [1]:
import pandas as pd
import re
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch



/Users/tejus/.pyenv/versions/3.8.5/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
comments_df = pd.read_csv('../data/YTcomments.csv')
comments_df.head()

,Comment
0,Copy nothing? More like understand nothing.
1,De macho Alfa lomo plateado pelo en pecho a co...
2,Où sont les voitures ? Cest quoi cest guignols...
3,This thing is just a waste of time where is th...
4,WTF did I just watch?


Let the cleaning begin!!
1) get rid of all non-english comments 
2) ill come up with more stuff 

TooManyRequests: 429 Too Many Requests
Usage cap exceeded: Monthly product cap